In [1]:
#####ベイジアン非負値行列因子分解#####
##ライブラリを読み込み
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy
import scipy.stats as ss
from numpy.random import *
from scipy import optimize
from scipy.stats import norm

In [2]:
####データの発生####
#np.random.seed(8742)   #シードを設定

#データの設定
hh = 3000   #ユーザー数
item = 500   #アイテム数
k = 10   #基底数

In [3]:
##非負値行列因子分解の仮定に基づきデータを生成
#ガンマ分布よりパラメータを生成
alpha01 = 0.2; beta01 = 1.0
alpha02 = 0.15; beta02 = 0.8
W0 = numpy.random.gamma(alpha01, int(1/beta01), hh*k).reshape(hh, k)
H0 = numpy.random.gamma(alpha02, int(1/beta02), item*k).reshape(k, item)
WH = np.dot(W0, H0)

In [4]:
#ポアソン分布よりデータを生成
Data = np.zeros((hh, item))
for j in range(item):
    Data[:, j] = numpy.random.poisson(WH[:, j], hh)

#購買数を確認
print(np.round(np.sum(Data, axis=0), 0))
print(np.round(np.sum(Data, axis=1), 0))

#ベストな対数尤度
LLbest = np.sum(scipy.stats.poisson.logpmf(Data, WH))
print(LLbest)

[   75.   519.  1498.   884.   787.  2218.  3678.   239.   898.   127.
  2599.  1086.   323.   272.   433.   429.   581.    64.   434.   112.
   618.   548.  1508.  1412.   164.   181.   819.   957.   245.  2003.
   214.   860.  1588.   639.   244.   780.  1744.   464.  1245.   545.
   631.   853.  1676.  1983.   551.    70.   247.  2518.   108.  2126.
   360.   617.  1933.  1387.   713.   549.   966.  1045.   445.  2727.
   951.  1466.  1548.   803.   470.   277.   755.  1377.  1092.   755.
    22.   864.   165.  2261.   439.   440.    83.   675.  1578.   223.
    53.   780.   229.  1513.  1001.   490.   172.   168.   780.   837.
  1128.   731.   456.   389.   459.   533.   741.  1338.   196.  1188.
   172.  2056.  1161.  1613.  1032.  1448.  1192.   497.  1154.   176.
   329.   607.   388.   494.    88.   324.   459.   505.   569.   533.
   415.    62.  1163.  1619.  2476.   899.   256.  1384.   239.   722.
   563.   995.    77.   196.   732.   533.   409.   340.   346.  1546.
  1218

In [269]:
####マルコフ連鎖モンテカルロ法で非負値行列因子分解を推定####
##アルゴリズムの設定
R = 5000
keep = 2
disp = 10
burnin = int(1000/keep)

##事前分布の設定
alpha1 = 0.01; beta1 = 0.01
alpha2 = 0.01; beta2 = 0.01

##初期値の設定
W = numpy.random.gamma(0.1, 1/0.1, hh*k).reshape(hh, k)
H = numpy.random.gamma(0.1, 1/0.1, item*k).reshape(k, item)

##サンプリング結果の保存用配列
W_array = np.zeros((hh, k, int(R/keep)))
H_array = np.zeros((k, item, int(R/keep)))
LAMBDA = np.zeros((hh, item))

In [270]:
####ギブスサンプリングでパラメータをサンプリング####
for rp in range(R):
    
    ##ガンマ分布からWをサンプリング
    WH = np.dot(W, H)
    Lambda = np.zeros((hh, item, k))
    #補助変数lambdaを更新
    for j in range(k):
        Lambda[:, :, j] = np.dot(W[:, j].reshape(hh, 1), H[j, :].reshape(1, item)) / WH

    #ガンマ分布からパラメータを生成
    for j in range(k):
        w1 = alpha1 + np.sum(Lambda[:, :, j] * Data, axis=1)
        w2 = beta1 + sum(H[j, :])
        W[:, j] = numpy.random.gamma(w1, 1/w2, hh) 

    #各列ベクトルの要素を正規化
    W = W / np.sum(W, axis=0).repeat(hh).reshape(k, hh).T * hh/5


    ##ガンマ分布よりHをサンプリング
    WH = np.dot(W, H)
    Lambda = np.zeros((hh, item, k))
    #補助変数lambdaを更新
    for j in range(k):
        Lambda[:, :, j] = np.dot(W[:, j].reshape(hh, 1), H[j, :].reshape(1, item)) / WH

    #ガンマ分布からパラメータを生成
    for j in range(k):
        h1 = alpha1 + np.sum(Lambda[:, :, j] * Data, axis=0)
        h2 = beta1 + sum(W[:, j])
        H[j, :] = numpy.random.gamma(h1, 1/h2, item) 


    ##サンプリング結果の格納と表示
    if rp%keep==0:
        mkeep = int(rp/keep)
        W_array[:, :, mkeep] = W
        H_array[:, :, mkeep] = H

    if rp%disp==0:
        LL = np.sum(scipy.stats.poisson.logpmf(Data, WH))   #非負値行列因子分解の対数尤度
        print(rp)
        print(np.round(np.array((LL, LLbest)), 1))

0
[-3227650.3  -708663. ]
10
[-902066.8 -708663. ]
20
[-873010.6 -708663. ]
30
[-856824.6 -708663. ]
40
[-842073.3 -708663. ]
50
[-827563.5 -708663. ]
60
[-812227.7 -708663. ]
70
[-799664.4 -708663. ]


KeyboardInterrupt: 

array([-886611.3, -709941.4])

array([[-0.0233711 , -0.29913695, -0.04385204, ..., -2.3449761 ,
        -0.12098072, -0.21637346],
       [-1.08195989, -0.71632631, -0.09320898, ..., -0.21105694,
        -1.30729323, -1.71476141],
       [-0.0407584 , -0.38928373, -0.01054559, ..., -0.02768325,
        -0.07608852, -1.11797549],
       ..., 
       [-0.04508845, -0.70308476, -0.13833732, ..., -0.18364243,
        -0.63322439, -1.09316311],
       [-0.05620867, -2.5570406 , -0.02323603, ..., -0.0597573 ,
        -0.16449201, -2.1906021 ],
       [-0.0609607 , -0.47437417, -0.03582721, ..., -0.10243519,
        -0.1641441 , -0.22614619]])